In [ ]:
import numpy as np
import pandas as pd
import yaml
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.callbacks import EarlyStopping
import os
import sys
import warnings
from dotenv import load_dotenv

warnings.filterwarnings("ignore")
load_dotenv()
REPO_PATH = os.getenv("REPO_PATH")
sys.path.insert(0, rf'{REPO_PATH}src_HF')

from utils.forecast_utils import RNNGenerator, build_rnn_model

### Create RNN data generator

In [ ]:

# load yaml file
with open('variable_config.yaml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

# Define feature list and target
feature_columns = list()
feature_columns += config['BASE']
feature_columns += config['SENTIMENT']
feature_columns += config['TEMPORAL']
feature_columns += config['LAGS']


### Build and fit model

In [ ]:
data_params = {
    'feature_columns': feature_columns,
    'target_column': 'TARGET',
    'window_size': 30,
    'test_size': 0.2,
    'val_size': 0.2
}

model_params = {
    'units_first_layer': 64,
    'units_second_layer': 64,
    'dropout_rate_first': 0.1,
    'dropout_rate_second': 0.1,
    'l2_strength': 1e-05,
    'learning_rate': 0.01,
    'batch_size': 32,
    'noise_std': 0.01
}

rnn_type='BiGRU'

# Specify data
gen = RNNGenerator(future='CLc1', topic='CRU')

gen.preprocess_data(
    data_params['feature_columns'], 
    data_params['target_column'], 
    data_params['window_size'],
    test_size=data_params['test_size'],
    val_size=data_params['val_size']
)

# Build the model
model = build_rnn_model(
    rnn_type, 
    model_params, 
    (data_params['window_size'], len(data_params['feature_columns']))
)

# Configure early stopping
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    restore_best_weights=True, 
    verbose=1
)

# Train the model with early stopping
history = model.fit(
    gen.train_generator, 
    epochs=150,
    batch_size=model_params['batch_size'], 
    validation_data=gen.val_generator, 
    callbacks=[early_stopping], 
    verbose=1
)

current_dt = pd.Timestamp.now().strftime('%Y.%m.%d_%H.%M')

fig, ax = plt.subplots(figsize=(8, 5), dpi=200)
ax.plot(history.history['loss'], label='Train Loss')
ax.plot(history.history['val_loss'], label='Validation Loss')
ax.set_title('Model Training and Validation Loss')
ax.set_xlabel('Epochs')
ax.set_ylabel('Loss')
ax.legend()

### Save model

In [ ]:
identifier: str = '1' + '_'

# save model
model_name = f'{gen.future}_{gen.topic}_{rnn_type}_{identifier}{current_dt}'

# check if file model is not already in the folder
if not os.path.exists(f'model_archive/{model_name}.h5'):
    model.save(f'model_archive/{model_name}.h5')


In [ ]:
# Evaluate the model
view = 500

test_predictions = model.predict(gen.test_generator)
test_predictions = test_predictions.flatten()

test_targets = np.concatenate([y for _, y in gen.test_generator])
test_targets = test_targets.flatten()

mse = mean_squared_error(test_targets, test_predictions)
mae = mean_absolute_error(test_targets, test_predictions)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')

# plot fit vs actual
plt.figure(figsize=(10, 5))
plt.plot(test_targets[-view:], label='Actual')
plt.plot(test_predictions[-view:], label='Predicted')
plt.title('Model Fit vs Actual')
plt.xlabel('Samples')
plt.ylabel('Price')
plt.legend()

In [ ]:
plt.figure(figsize=(10, 5))
residuals = test_targets - test_predictions
plt.plot(residuals[-view:], color='blue')
plt.title('Residuals of Model Predictions', fontsize=16)
plt.xlabel('Time Steps', fontsize=15)
plt.ylabel('Residuals', fontsize=15)
plt.show()